In [1]:
import numpy as np
import pandas as pd

In [96]:
nba = pd.read_csv('nba.csv')
nba.head()

,date,start_time,vis_team,home_team,vis_points,home_points,ot,attendance,home_initial,vis_initial,home_players,vis_players,referees,exp_score_home,exp_score_vis
0,2019-10-22,20:00:00,New Orleans Pelicans,Toronto Raptors,122,130,1,20787,TOR,NOP,"Kyle Lowry,Fred VanVleet,Pascal Siakam,OG Anun...","Jrue Holiday,Brandon Ingram,J.J. Redick,Lonzo ...","Tony Brown,Eric Dalen,Josh Tiven",79.329620,80.353980
1,2019-10-22,22:30:00,Los Angeles Lakers,Los Angeles Clippers,102,112,0,19068,LAC,LAL,"Kawhi Leonard,Patrick Beverley,Landry Shamet,P...","Anthony Davis,LeBron James,Danny Green,Avery B...","Kane Fitzgerald,Mark Lindsay,Dedric Taylor",57.011597,76.038714
2,2019-10-23,19:00:00,Chicago Bulls,Charlotte Hornets,125,126,0,15424,CHO,CHI,"P.J. Washington,Cody Zeller,Dwayne Bacon,Miles...","Lauri Markkanen,Zach LaVine,Wendell Carter,Ott...","Rodney Mott,Gediminas Petraitis,Leroy Richardson",60.023165,62.076264
3,2019-10-23,19:00:00,Detroit Pistons,Indiana Pacers,119,110,0,17923,IND,DET,"Myles Turner,Malcolm Brogdon,Domantas Sabonis,...","Andre Drummond,Tony Snell,Bruce Brown,Markieff...","Marc Davis,Brett Nansel,Ben Taylor",77.901700,57.526521
4,2019-10-23,19:00:00,Cleveland Cavaliers,Orlando Magic,85,94,0,18846,ORL,CLE,"Nikola Vučević,Aaron Gordon,Evan Fournier,D.J....","Kevin Love,Tristan Thompson,Darius Garland,Col...","Brent Barnaky,Eric Lewis,Haywoode Workman",75.108187,73.763983


# Calculate recovery time from previous match

In [107]:
nba['date'] = nba['date'].apply(pd.to_datetime)

In [124]:
teams = ['ATL', 'BRK', 'BOS', 'CHO', 'CHI', 'CLE', 'DAL', 'DEN', 'DET', \
       'GSW', 'HOU', 'IND', 'LAC', 'LAL', 'MEM', 'MIA', 'MIL', 'MIN', \
       'NOP', 'NYK', 'OKC', 'ORL', 'PHI', 'PHO', 'POR', 'SAC', 'SAS', \
       'TOR', 'UTA', 'WAS']

In [185]:
nba['recovery_time'] = 0

,date,start_time,vis_team,home_team,vis_points,home_points,ot,attendance,home_initial,vis_initial,home_players,vis_players,referees,exp_score_home,exp_score_vis,recovery_time
0,2019-10-22,20:00:00,New Orleans Pelicans,Toronto Raptors,122,130,1,20787,TOR,NOP,"Kyle Lowry,Fred VanVleet,Pascal Siakam,OG Anun...","Jrue Holiday,Brandon Ingram,J.J. Redick,Lonzo ...","Tony Brown,Eric Dalen,Josh Tiven",79.329620,80.353980,0
1,2019-10-22,22:30:00,Los Angeles Lakers,Los Angeles Clippers,102,112,0,19068,LAC,LAL,"Kawhi Leonard,Patrick Beverley,Landry Shamet,P...","Anthony Davis,LeBron James,Danny Green,Avery B...","Kane Fitzgerald,Mark Lindsay,Dedric Taylor",57.011597,76.038714,0
2,2019-10-23,19:00:00,Chicago Bulls,Charlotte Hornets,125,126,0,15424,CHO,CHI,"P.J. Washington,Cody Zeller,Dwayne Bacon,Miles...","Lauri Markkanen,Zach LaVine,Wendell Carter,Ott...","Rodney Mott,Gediminas Petraitis,Leroy Richardson",60.023165,62.076264,0
3,2019-10-23,19:00:00,Detroit Pistons,Indiana Pacers,119,110,0,17923,IND,DET,"Myles Turner,Malcolm Brogdon,Domantas Sabonis,...","Andre Drummond,Tony Snell,Bruce Brown,Markieff...","Marc Davis,Brett Nansel,Ben Taylor",77.901700,57.526521,0
4,2019-10-23,19:00:00,Cleveland Cavaliers,Orlando Magic,85,94,0,18846,ORL,CLE,"Nikola Vučević,Aaron Gordon,Evan Fournier,D.J....","Kevin Love,Tristan Thompson,Darius Garland,Col...","Brent Barnaky,Eric Lewis,Haywoode Workman",75.108187,73.763983,0


In [186]:
for team in teams:
    game_date = nba[(nba['home_initial'] == team) | (nba['vis_initial'] == team)].date
    date_diff = (game_date - game_date.shift()).dt.days
    nba['recovery_time'] += date_diff.reindex(list(range(nba.index.min(),nba.index.max()+1)),fill_value=0)
    

ATL
BRK
BOS
CHO
CHI
CLE
DAL
DEN
DET
GSW
HOU
IND
LAC
LAL
MEM
MIA
MIL
MIN
NOP
NYK
OKC
ORL
PHI
PHO
POR
SAC
SAS
TOR
UTA
WAS


# Calculate recent performance

In [217]:
nba['perform'] = 0
for team in teams:
    home_perform = nba[nba.home_initial == team].home_points - nba[nba.home_initial == team].vis_points
    away_perform = nba[nba.vis_initial == team].vis_points - nba[nba.vis_initial == team].home_points
    home_perform = home_perform.reindex(list(range(nba.index.min(),nba.index.max()+1)),fill_value=0)
    away_perform = away_perform.reindex(list(range(nba.index.min(),nba.index.max()+1)),fill_value=0)
    perform = home_perform + away_perform
    perform = perform[perform != 0]
    roll_perform = abs(perform).rolling(3).mean()
    nba['perform'] += roll_perform.reindex(list(range(nba.index.min(),nba.index.max()+1)),fill_value=0)

In [218]:
nba.sample(10)

,date,start_time,vis_team,home_team,vis_points,home_points,ot,attendance,home_initial,vis_initial,home_players,vis_players,referees,exp_score_home,exp_score_vis,recovery_time,perform
232,2019-11-23,20:30:00,Detroit Pistons,Milwaukee Bucks,90,104,0,17585,MIL,DET,"Eric Bledsoe,Giannis Antetokounmpo,Brook Lopez...","Langston Galloway,Andre Drummond,Markieff Morr...","Scott Foster,Courtney Kirkland,Justin Van Duyne",73.001247,62.753304,3.0,29.666667
495,2019-12-30,22:00:00,Phoenix Suns,Portland Trail Blazers,122,116,0,19896,POR,PHO,"Damian Lillard,CJ McCollum,Hassan Whiteside,Ca...","Kelly Oubre,Devin Booker,Ricky Rubio,Aron Bayn...","James Capers Jr.,Evan Scott,Scott Twardoski",91.655607,60.721364,4.0,12.333333
470,2019-12-27,22:30:00,Phoenix Suns,Golden State Warriors,96,105,0,18064,GSW,PHO,"Damion Lee,D'Angelo Russell,Glenn Robinson,Dra...","Devin Booker,Ricky Rubio,Aron Baynes,Kelly Oub...","Pat Fraher,Phenizee Ransom,James Williams",51.007836,60.721364,6.0,18.333333
407,2019-12-18,19:00:00,Toronto Raptors,Detroit Pistons,112,99,0,15319,DET,TOR,"Andre Drummond,Blake Griffin,Luke Kennard,Tony...","Kyle Lowry,Pascal Siakam,OG Anunoby,Norman Pow...","Ashley Moyer-Gleich,Scott Wall,Tom Washington",65.952704,78.111817,4.0,25.333333
177,2019-11-16,20:00:00,New Orleans Pelicans,Miami Heat,94,109,0,19600,MIA,NOP,"Kendrick Nunn,Jimmy Butler,Bam Adebayo,Duncan ...","E'Twaun Moore,Kenrich Williams,Jrue Holiday,De...","Brent Barnaky,Leroy Richardson,Zach Zarba",71.485274,46.714150,4.0,20.333333
10,2019-10-23,21:00:00,Oklahoma City Thunder,Utah Jazz,95,100,0,18306,UTA,OKC,"Rudy Gobert,Donovan Mitchell,Royce O'Neale,Mik...","Shai Gilgeous-Alexander,Chris Paul,Danilo Gall...","Kane Fitzgerald,Tre Maddox,Dedric Taylor",79.588229,71.296102,NaN,NaN
284,2019-12-01,15:30:00,Memphis Grizzlies,Minnesota Timberwolves,115,107,0,12276,MIN,MEM,"Karl-Anthony Towns,Andrew Wiggins,Robert Covin...","Jae Crowder,Dillon Brooks,Tyus Jones,Jaren Jac...","John Butler,Pat Fraher,Courtney Kirkland",74.647385,45.557772,6.0,17.000000
556,2020-01-08,19:00:00,Washington Wizards,Orlando Magic,89,123,0,16013,ORL,WAS,"Markelle Fultz,Khem Birch,Nikola Vučević,Aaron...","Isaac Bonga,Isaiah Thomas,Gary Payton,Jordan M...","Mitchell Ervin,Kevin Scott,Michael Smith",68.567116,35.747089,4.0,37.333333
155,2019-11-13,19:30:00,Washington Wizards,Boston Celtics,133,140,0,19156,BOS,WAS,"Marcus Smart,Kemba Walker,Jaylen Brown,Jayson ...","Bradley Beal,Isaiah Thomas,Thomas Bryant,Troy ...","Brent Barnaky,Mitchell Ervin,Tre Maddox",86.827883,68.214976,7.0,24.000000
352,2019-12-10,20:00:00,Atlanta Hawks,Miami Heat,121,135,1,19600,MIA,ATL,"Bam Adebayo,Duncan Robinson,Kendrick Nunn,Jimm...","Trae Young,De'Andre Hunter,Jabari Parker,Kevin...","Tony Brothers,Derrick Collins,Marat Kogut",71.485274,74.427128,4.0,23.000000


In [214]:
abs(perform).rolling(3).mean()

15           NaN
35           NaN
46     19.333333
64     16.666667
75     13.000000
82     11.333333
95     10.666667
105    10.666667
119    10.666667
130     9.666667
143     8.666667
158    15.333333
172    15.000000
186    13.000000
197    10.666667
205    25.000000
222    23.666667
249    18.333333
265     7.000000
273    11.333333
289    11.666667
294    15.333333
305    14.666667
323    14.000000
350     8.333333
365     4.000000
376     6.000000
392    10.333333
414    13.000000
428    11.666667
453     7.666667
456     8.333333
470    10.000000
479    13.666667
500    11.000000
511    13.000000
530     8.666667
546    11.666667
561     9.666667
575    10.333333
586    12.666667
601    18.666667
616    16.666667
632    14.666667
651     7.333333
664    17.333333
678    16.333333
699    18.333333
715    12.333333
731    13.333333
743    15.000000
759    12.666667
774     9.333333
798    12.000000
dtype: float64

In [209]:
def past_perform(val):
    if val < 0: return 0
    elif val > 0: return 1
    else: pass
    
perform = perform.apply(win_or_lose)
perform

15     0
35     0
46     1
64     0
75     0
82     0
95     1
105    0
119    0
130    0
143    0
158    0
172    0
186    0
197    1
205    0
222    0
249    0
265    1
273    0
289    0
294    0
305    0
323    1
350    0
365    0
376    0
392    0
414    0
428    1
453    1
456    1
470    1
479    0
500    0
511    0
530    0
546    0
561    0
575    0
586    0
601    0
616    0
632    1
651    0
664    0
678    0
699    0
715    0
731    0
743    1
759    0
774    1
798    0
dtype: int64

In [207]:
perform.iloc[-3:]

759    -8
774     4
798   -24
dtype: int64